In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.7 MB/s eta 0:00:00


In [ ]:
import os
import math
import random
import regex as re
import pandas as pd
import numpy as np
from typing import Optional, Sequence

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, f1_score

from tqdm import tqdm
import torch
from torch import nn
from torch import Tensor
from torch.nn import functional as F
from torch.optim.lr_scheduler import _LRScheduler
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer, EarlyStoppingCallback, AutoModel, AutoConfig, T5TokenizerFast, T5ForConditionalGeneration, ElectraTokenizerFast, ElectraModel
from transformers import BertTokenizerFast, BertModel

import gc
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving sample_submission.csv to sample_submission.csv
Saving test.csv to test.csv
Saving train.csv to train.csv


In [ ]:
train = pd.read_csv('train.csv').drop(['ID'], axis=1)
test = pd.read_csv('test.csv')

In [ ]:
train['문장'] = train['문장'].apply(lambda x: re.sub("[^ A-Za-z0-9가-힣]", "", x))
train['문장'] = train['문장'].apply(lambda x: re.sub("[ +]", " ", x))

test['문장'] = test['문장'].apply(lambda x: re.sub("[^ A-Za-z0-9가-힣]", "", x))
test['문장'] = test['문장'].apply(lambda x: re.sub("[ +]", " ", x))

In [ ]:
# https://github.com/catSirup/KorEDA/blob/master/eda.py
def swap_word(new_words):
    random_idx_1 = random.randint(0, len(new_words)-1)
    random_idx_2 = random_idx_1
    counter = 0

    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words)-1)
        counter += 1
        if counter > 3:
            return new_words

    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
    return new_words

def random_swap(words, n):
    new_words = words.copy()
    for _ in range(n):
        new_words = swap_word(new_words)
    return new_words

def text_aug(sentence, alpha_rs = 0.1, num_aug=3):
    words = sentence.split(' ')
    words = [word for word in words if word != ""]
    num_words = len(words)

    augmented_sentences = []
    num_new_per_technique = num_aug

    n_rs = max(1, int(alpha_rs*num_words))

    for _ in range(num_new_per_technique):
        a_words = random_swap(words, n_rs)
        augmented_sentences.append(" ".join(a_words))

    augmented_sentences = [sentence for sentence in augmented_sentences]
    random.shuffle(augmented_sentences)

    if num_aug >= 1:
        augmented_sentences = augmented_sentences[:num_aug]
    else:
        keep_prob = num_aug / len(augmented_sentences)
        augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]
    return augmented_sentences

aug = train['문장'].apply(lambda x: text_aug(x))

In [ ]:
tmp1 = train.copy()
tmp1['문장'] = list(map(lambda x: x[0], aug))

tmp2 = train.copy()
tmp2['문장'] = list(map(lambda x: x[1], aug))

tmp3 = train.copy()
tmp3['문장'] = list(map(lambda x: x[2], aug))

In [ ]:
train = pd.concat([train,tmp1,tmp2,tmp3]).drop_duplicates(keep='first').sample(frac=1).reset_index(drop=True)
train

,문장,유형,극성,시제,확실성,label
0,8일 민주노총 화학섬유식품산업노조 웹젠지회는 이날 임금협약 사옥 1층 대회의실에서 ...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
1,세계 최대 라임이 업체 공유킥보드 한국에서 서비스를 잠정 중단한다,사실형,긍정,미래,확실,사실형-긍정-미래-확실
2,한국전쟁 일이 한인커뮤니티라는 두 매개를 활용하면 USA250 사업에서 우리의 할 ...,추론형,긍정,미래,불확실,추론형-긍정-미래-불확실
3,올해 평검사 정기인사는 2월 정해졌다 3일로,사실형,긍정,과거,확실,사실형-긍정-과거-확실
4,이어 김 대표의 선임으로 글로벌 시장에서 펍지의 성장과 배틀그라운드 지식재산권IP의...,사실형,긍정,과거,불확실,사실형-긍정-과거-불확실
...,...,...,...,...,...,...
64722,이후 지방과 견갑골을 이뤄진다 장기 적체가 제거하고,사실형,긍정,현재,확실,사실형-긍정-현재-확실
64723,정부의 다주택자 양도세 중과 유예 조치에 맞춰 절세 효과를 노린 매도 물량이 쌓이면...,사실형,긍정,현재,확실,사실형-긍정-현재-확실
64724,지금은 수많은 수익화할 크리에이터들이 콘텐츠를 만드는 콘텐츠의 뿐만 하지만 크리에이...,사실형,긍정,현재,확실,사실형-긍정-현재-확실
64725,오는 6월에는 2년 노선과 인천로마 노선을 인천파리 3개월 만에 운항 재개한다,사실형,긍정,미래,확실,사실형-긍정-미래-확실


In [ ]:
train['문장'].str.len().max(), test['문장'].str.len().max()

(496, 378)

In [ ]:
device = torch.device("cuda")
model_path = "snunlp/KR-BERT-char16424"
tokenizer = AutoTokenizer.from_pretrained(model_path)
length = train['문장'].str.len().max()

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            st_type = self.labels['type'][idx]
            st_polarity = self.labels['polarity'][idx]
            st_tense = self.labels['tense'][idx]
            st_certainty = self.labels['certainty'][idx]
            item["labels"] = torch.tensor(st_type), torch.tensor(st_polarity), torch.tensor(st_tense), torch.tensor(st_certainty)
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
config=AutoConfig.from_pretrained(model_path)
config._name_or_path = 'kr.kim'
print(config.num_hidden_layers)
config

12


BertConfig {
  "_name_or_path": "kr.kim",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.28.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 16424
}

In [ ]:
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        if model_path == 'snunlp/KR-BERT-char16424':
          config.attention_type = "original_full"
        self.base_model = AutoModel.from_pretrained(model_path, config=config)
        self.out = self.base_model.encoder.layer[-1].output.dense.out_features//2
        self.norm = 384
        
        self.Linear1 = nn.Sequential(
            nn.Linear(768, 384),
            nn.BatchNorm1d(self.norm))
        self.Linear2 = nn.Sequential(
            nn.Linear(768, 384),
            nn.BatchNorm1d(self.norm))
        self.Linear3 = nn.Sequential(
            nn.Linear(768, 384),
            nn.BatchNorm1d(self.norm))
        self.Linear4 = nn.Sequential(
            nn.Linear(768, 384),
            nn.BatchNorm1d(self.norm))
        
        self.type_classifier = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=self.out, out_features=4),
        )
        self.polarity_classifier = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=self.out, out_features=3),
        )
        self.tense_classifier = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=self.out, out_features=3),
        )
        self.certainty_classifier = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=self.out, out_features=2),
        )

        #모델이 과적합(overfitting)되지 않도록 하기 위해 일부 뉴런을 임의로 끄기 위해 Dropout 레이어 사용

        #출력층: 4개의 분류기 (type_classifier, polarity_classifier, tense_classifier, certainty_classifier)
    def forward(self, input_ids, attention_mask, labels=None, token_type_ids=None):
        x = self.base_model(input_ids=input_ids, attention_mask=attention_mask)[0]
        x = x[:,0,:]

        out4 = self.Linear4(x)
        certainty_output = self.certainty_classifier(out4)
        
        out3 = self.Linear3(x)
        tense_output = self.tense_classifier((out3+out4))
        
        out2 = self.Linear2(x)
        polarity_output = self.polarity_classifier((out2+out3+out4))
        
        out1 = self.Linear1(x) 
        type_output = self.type_classifier((out1+out2+out3+out4))
        
        return type_output, polarity_output, tense_output, certainty_output

        #입력층: input_ids, attention_mask, token_type_ids (BERT의 입력에 필요한 세 가지 입력)

In [ ]:
lr = 1e-4
stop = 3
epoch = 100
batch = 4
seed = 42

In [ ]:
#Focal Loss는 불균형한 데이터셋에서 분류 문제를 해결하는 손실 함수입니다. 특히, 이 손실 함수는 희귀 클래스와 다수 클래스 간의 균형을 맞추는 데에 효과적입니다.
#Focal Loss는 희귀한 샘플에 대한 예측이 잘못된 경우에만 가중치를 적용하도록 조정됩니다. 이렇게 하면 모델이 희귀한 샘플에 더 집중하도록 유도할 수 있습니다.

#Weighted Cross-Entropy Loss
#Label Smoothing Loss
#Focal Tversky Loss

class FocalLoss(nn.Module):
    """ Focal Loss, as described in https://arxiv.org/abs/1708.02002.
    It is essentially an enhancement to cross entropy loss and is
    useful for classification tasks when there is a large class imbalance.
    x is expected to contain raw, unnormalized scores for each class.
    y is expected to contain class labels.
    Shape:
        - x: (batch_size, C) or (batch_size, C, d1, d2, ..., dK), K > 0.
        - y: (batch_size,) or (batch_size, d1, d2, ..., dK), K > 0.
    """

    def __init__(self,
                 alpha: Optional[Tensor] = None,
                 gamma: float = 0.,
                 reduction: str = 'mean',
                 ignore_index: int = -100):
        """Constructor.
        Args:
            alpha (Tensor, optional): Weights for each class. Defaults to None.
            gamma (float, optional): A constant, as described in the paper.
                Defaults to 0.
            reduction (str, optional): 'mean', 'sum' or 'none'.
                Defaults to 'mean'.
            ignore_index (int, optional): class label to ignore.
                Defaults to -100.
        """
        if reduction not in ('mean', 'sum', 'none'):
            raise ValueError(
                'Reduction must be one of: "mean", "sum", "none".')

        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.ignore_index = ignore_index
        self.reduction = reduction

        self.nll_loss = nn.NLLLoss(
            weight=alpha, reduction='none', ignore_index=ignore_index)

    def __repr__(self):
        arg_keys = ['alpha', 'gamma', 'ignore_index', 'reduction']
        arg_vals = [self.__dict__[k] for k in arg_keys]
        arg_strs = [f'{k}={v!r}' for k, v in zip(arg_keys, arg_vals)]
        arg_str = ', '.join(arg_strs)
        return f'{type(self).__name__}({arg_str})'

    def forward(self, x: Tensor, y: Tensor) -> Tensor:
        if x.ndim > 2:
            # (N, C, d1, d2, ..., dK) --> (N * d1 * ... * dK, C)
            c = x.shape[1]
            x = x.permute(0, *range(2, x.ndim), 1).reshape(-1, c)
            # (N, d1, d2, ..., dK) --> (N * d1 * ... * dK,)
            y = y.view(-1)

        unignored_mask = y != self.ignore_index
        y = y[unignored_mask]
        if len(y) == 0:
            return torch.tensor(0.)
        x = x[unignored_mask]

        # compute weighted cross entropy term: -alpha * log(pt)
        # (alpha is already part of self.nll_loss)
        log_p = F.log_softmax(x, dim=-1)
        ce = self.nll_loss(log_p, y)

        # get true class column from each row
        all_rows = torch.arange(len(x))
        log_pt = log_p[all_rows, y]

        # compute focal term: (1 - pt)^gamma
        pt = log_pt.exp()
        focal_term = (1 - pt)**self.gamma

        # the full loss: -alpha * ((1 - pt)^gamma) * log(pt)
        loss = focal_term * ce

        if self.reduction == 'mean':
            loss = loss.mean()
        elif self.reduction == 'sum':
            loss = loss.sum()

        return loss


def focal_loss(alpha: Optional[Sequence] = None,
               gamma: float = 0.,
               reduction: str = 'mean',
               ignore_index: int = -100,
               device='cpu',
               dtype=torch.float32) -> FocalLoss:
    """Factory function for FocalLoss.
    Args:
        alpha (Sequence, optional): Weights for each class. Will be converted
            to a Tensor if not None. Defaults to None.
        gamma (float, optional): A constant, as described in the paper.
            Defaults to 0.
        reduction (str, optional): 'mean', 'sum' or 'none'.
            Defaults to 'mean'.
        ignore_index (int, optional): class label to ignore.
            Defaults to -100.
        device (str, optional): Device to move alpha to. Defaults to 'cpu'.
        dtype (torch.dtype, optional): dtype to cast alpha to.
            Defaults to torch.float32.
    Returns:
        A FocalLoss object
    """
    if alpha is not None:
        if not isinstance(alpha, Tensor):
            alpha = torch.tensor(alpha)
        alpha = alpha.to(device=device, dtype=dtype)

    fl = FocalLoss(
        alpha=alpha,
        gamma=gamma,
        reduction=reduction,
        ignore_index=ignore_index)
    return fl

#ASL(Amherst Symbolic Loss)를 구현한 PyTorch 모듈입니다. 이 손실 함수는 single-label classification 문제에 대한 손실 함수입니다.

class ASLSingleLabel(nn.Module):
    '''
    This loss is intended for single-label classification problems
    '''
    def __init__(self, gamma_pos=0, gamma_neg=4, eps: float = 0.1, reduction='mean'):
        super(ASLSingleLabel, self).__init__()

        self.eps = eps
        self.logsoftmax = nn.LogSoftmax(dim=-1)
        self.targets_classes = []
        self.gamma_pos = gamma_pos
        self.gamma_neg = gamma_neg
        self.reduction = reduction

#forward함수: 입력값 inputs와 정답 라벨 target을 받아서, logsoftmax를 적용한 후, targets_classes 리스트를 만듭니다. 
#이 리스트는 one-hot encoding으로, 정답 클래스를 제외한 모든 클래스의 값은 0이며, 정답 클래스만 1인 형태입니다.

    def forward(self, inputs, target):
        '''
        "input" dimensions: - (batch_size,number_classes)
        "target" dimensions: - (batch_size)
        '''
        num_classes = inputs.size()[-1]
        log_preds = self.logsoftmax(inputs)
        self.targets_classes = torch.zeros_like(inputs).scatter_(1, target.long().unsqueeze(1), 1)

        # ASL weights
        targets = self.targets_classes
        anti_targets = 1 - targets
        xs_pos = torch.exp(log_preds)
        xs_neg = 1 - xs_pos
        xs_pos = xs_pos * targets
        xs_neg = xs_neg * anti_targets
        asymmetric_w = torch.pow(1 - xs_pos - xs_neg,
                                 self.gamma_pos * targets + self.gamma_neg * anti_targets)
        log_preds = log_preds * asymmetric_w


        #ASL에서 사용하는 asymmetric weight를 계산합니다. 이 값은 정답 클래스에 대한 가중치는 gamma_pos 값에 따라서, 오답 클래스에 대한 가중치는 gamma_neg 값에 따라서 계산됩니다.
        # 이 가중치를 이용하여, log_preds 값을 가중치만큼 곱해줍니다.

#targets_classes에 대한 label smoothing을 적용하고, cross-entropy loss를 계산합니다. 이 때, reduction 인자가 'mean'이면, batch 내의 모든 샘플에 대한 loss의 평균을 구합니다.

        if self.eps > 0:  # label smoothing
            self.targets_classes = self.targets_classes.mul(1 - self.eps).add(self.eps / num_classes)

        # loss calculation
        loss = - self.targets_classes.mul(log_preds)

        loss = loss.sum(dim=-1)
        if self.reduction == 'mean':
            loss = loss.mean()

        return loss
        
def compute_metrics(pred):
    # label = [[cls1,cls2,...],]
    # preds = n list
    focal_loss = FocalLoss()
    labels = pred.label_ids
    preds = pred.predictions
    f1 = []
    focal = []
    for i in range(4):
        # focal.append(focal_loss(torch.tensor(preds[i], dtype=torch.float), torch.tensor(labels[::, i],dtype=torch.float)))
        f1.append(f1_score(y_true = labels[::, i], y_pred = preds[i], average='weighted'))
    return {
        #'focal': sum(focal),
        'f1-sum': sum(f1)/4
    }

    #transformers 라이브러리의 Trainer 클래스에서 모델 평가 시 사용되는 metrics 함수입니다. 입력으로 모델의 예측값(pred)을 받습니다.

    #함수는 각 레이블(4개)에 대해 F1-score를 계산하고, 이들의 평균값을 구하여 'f1-sum'이라는 이름의 딕셔너리를 반환합니다.
    # 이 때 F1-score 계산 시, y_true 값은 실제 레이블값(labels)을, y_pred 값은 모델의 예측값(preds)을 사용하며, 
    #average='weighted' 인자는 각 레이블별 샘플 수를 고려하여 가중평균을 계산한다는 의미

In [ ]:
# Define trainer
#손실 함수(compute_loss)를 사용자가 지정한 함수로 대체하여 학습을 진행할 수 있도록 구현

class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def compute_loss(self, model, inputs, return_outputs=False):
        # forward pass
        labels = inputs.pop("labels").to(torch.int64)
        
        type_logit, polarity_logit, tense_logit, certainty_logit = model(**inputs)
        

        # # focal loss
        criterion = {
            'type' : FocalLoss().to(device),
            'polarity' : FocalLoss().to(device),
            'tense' : FocalLoss().to(device),
            'certainty' : FocalLoss().to(device)
        }
        # # labels = labels.type(torch.float).clone().detach()
        loss = criterion['type'](type_logit, labels[::, 0]) + \
                    criterion['polarity'](polarity_logit, labels[::, 1]) + \
                    criterion['tense'](tense_logit, labels[::, 2]) + \
                    criterion['certainty'](certainty_logit, labels[::, 3])

        outputs = None, \
                    torch.argmax(type_logit, dim = 1), \
                    torch.argmax(polarity_logit, dim = 1),\
                    torch.argmax(tense_logit, dim = 1),\
                    torch.argmax(certainty_logit, dim = 1)
        return (loss, outputs) if return_outputs else loss
        '''
        # ASLoss
        criterion = {
            'type' : ASLSingleLabel().to(device),
            'polarity' : ASLSingleLabel().to(device),
            'tense' : ASLSingleLabel().to(device),
            'certainty' : ASLSingleLabel().to(device)
        }
        # labels = labels.type(torch.float).clone().detach()
        loss = criterion['type'](type_logit, labels[::, 0]) + \
                    criterion['polarity'](polarity_logit, labels[::, 1]) + \
                    criterion['tense'](tense_logit, labels[::, 2]) + \
                    criterion['certainty'](certainty_logit, labels[::, 3])

        outputs = None, \
                    torch.argmax(type_logit, dim = 1), \
                    torch.argmax(polarity_logit, dim = 1),\
                    torch.argmax(tense_logit, dim = 1),\
                    torch.argmax(certainty_logit, dim = 1)
        return (loss, outputs) if return_outputs else loss
        '''
#compute_loss 함수는 모델, 입력 데이터, return_outputs 옵션을 인자로 받습니다. return_outputs가 True로 설정되면 함수는 손실 값과 모델의 출력값들을 튜플 형태로 반환하고, 
#False로 설정되면 손실 값만 반환합니다.

#labels는 입력 데이터에서 제거되어 별도로 변수에 저장되고, inputs를 모델의 입력으로 사용합니다. 
#모델의 출력값들(type_logit, polarity_logit, tense_logit, certainty_logit)과 labels를 사용하여 4개의 손실 함수(nn.CrossEntropyLoss())를 정의하고, 
#이들을 모두 더한 값을 loss 변수에 저장합니다.

#마지막으로, 모델의 출력값들 중 가장 큰 값을 갖는 인덱스(torch.argmax)를 사용하여 각 레이블의 예측값(outputs)을 구하고, 손실 값과 함께 반환합니다.

In [ ]:
유형 = LabelEncoder()
유형.fit(train['유형'])

극성 = LabelEncoder()
극성.fit(train['극성'])

시제 = LabelEncoder()
시제.fit(train['시제'])

확실성 = LabelEncoder()
확실성.fit(train['확실성'])

def encoding(X_train, X_val):
    X_train['유형'] = 유형.transform(X_train['유형'])
    X_val['유형'] = 유형.transform(X_val['유형'])

    X_train['극성'] = 극성.transform(X_train['극성'])
    X_val['극성'] = 극성.transform(X_val['극성'])

    X_train['시제'] = 시제.transform(X_train['시제'])
    X_val['시제'] = 시제.transform(X_val['시제'])

    X_train['확실성'] = 확실성.transform(X_train['확실성'])
    X_val['확실성'] = 확실성.transform(X_val['확실성'])

    train_labels = {
        'type' : X_train['유형'].values,
        'polarity' : X_train['극성'].values,
        'tense' : X_train['시제'].values,
        'certainty' : X_train['확실성'].values
    }

    val_labels = {
        'type' : X_val['유형'].values,
        'polarity' : X_val['극성'].values,
        'tense' : X_val['시제'].values,
        'certainty' : X_val['확실성'].values
    }
    return train_labels, val_labels

    #이 코드는 LabelEncoder를 사용하여 카테고리형 데이터를 숫자형으로 변환하는 함수입니다.

#LabelEncoder는 sklearn의 preprocessing 모듈에 있는 클래스로, 카테고리형 데이터를 0부터 시작하는 숫자로 변환합니다.
# 예를 들어, ['dog', 'cat', 'dog', 'fish']와 같은 문자열 데이터를 [0, 1, 0, 2]와 같은 숫자 데이터로 변환합니다.

#위 코드에서는 train 데이터와 validation 데이터에서 각각 '유형', '극성', '시제', '확실성' 칼럼의 값을 LabelEncoder를 사용하여 변환합니다. 
#변환된 숫자 데이터는 각각 'type', 'polarity', 'tense', 'certainty' 라벨에 대응되어 딕셔너리 형태로 저장됩니다.

#이렇게 변환된 숫자 데이터를 모델의 입력값으로 사용할 수 있습니다.

이 코드는 K-fold 교차 검증을 수행하는 코드입니다. 주요 동작 과정은 다음과 같습니다.

1. AutoConfig.from_pretrained 함수를 사용하여 사전 학습된 모델의 설정 정보를 불러옵니다.
2. config.num_hidden_layers 값을 확인하고, 12로 설정합니다.
3. KFold 객체를 생성하여 데이터를 k개의 fold로 분할합니다.
4. k개의 fold에 대해 반복문을 실행하면서 다음을 수행합니다.
   1. 학습용 데이터와 검증용 데이터를 구분합니다.
   2. encoding 함수를 사용하여 각 데이터의 레이블을 인코딩합니다.
   3. tokenizer 함수를 사용하여 각 데이터의 텍스트를 토큰화합니다.
   4. CustomModel 클래스로부터 모델 객체를 생성합니다.
   5. TrainingArguments 클래스로부터 훈련 인수를 생성합니다.
   6. CustomTrainer 클래스로부터 트레이너 객체를 생성합니다.
   7. trainer.train() 함수를 호출하여 모델을 학습시킵니다.
   8. 메모리 관리를 위해 모델과 트레이너 객체를 삭제하고, GPU 메모리를 비웁니다.
위 코드에서 사용된 CustomModel, CustomDataset, CustomTrainer는 Hugging Face의 Transformers 라이브러리를 사용하여 PyTorch 모델을 구현하기 위한 클래스입니다. 이들 클래스는 사용자 정의 모델, 데이터셋, 트레이너를 만들기 위한 기반 클래스를 제공합니다. 이들 클래스에 대한 자세한 정보는 Hugging Face의 Transformers 문서에서 확인할 수 있습니다.

In [ ]:
config=AutoConfig.from_pretrained(model_path)
config._name_or_path = 'kr.kim'
print(f'hidden_layers : {config.num_hidden_layers}')
config.num_hidden_layers = 12
print(f'now_hidden_layers : {config.num_hidden_layers}')

kf = KFold(n_splits=5, random_state=seed, shuffle=True)
for i, (train_index, test_index) in enumerate(kf.split(train)):
    print(f'Round {i}')
    X_train, X_val = train.loc[train_index, :], train.loc[test_index, :]
    train_labels, val_labels = encoding(X_train, X_val)
    token_train, token_val = tokenizer(X_train.문장.tolist(), padding=True, truncation=True, max_length=length), tokenizer(X_val.문장.tolist(), padding=True, truncation=True, max_length=length)
    train_dataset, val_dataset = CustomDataset(token_train, train_labels), CustomDataset(token_val, val_labels)
    model = CustomModel()
    model.to(device)
    args = TrainingArguments(run_name = f'fold_{i}',                                # 모델이름
                             output_dir= f"fold_{i}",                               # 모델저장경로
                             evaluation_strategy="steps",                           # 모델의 평가를 언제 진행할지
                             eval_steps=100,                                        # 500 스텝 마다 모델 평가
                             save_steps=100,                                        # 500 스텝 마다 모델 저장
                             save_total_limit = 2,                                  # 저장할 모델의 갯수
                             logging_steps=100,                                     # 학습로스 로깅
                             per_device_train_batch_size=batch,                     # GPU에 학습데이터를 몇개씩 올려서 학습할지
                             per_device_eval_batch_size=batch,                      # GPU에 학습데이터를 몇개씩 올려서 평가할지
                             gradient_accumulation_steps=16,                        # 가상배치
                             num_train_epochs=epoch,                                # 전체 학습 진행 횟수
                             learning_rate=lr,                                      # 학습률 정의 
                             seed=seed,                                             # seed
                             load_best_model_at_end=True,                           # 평가기준 스코어가 좋은 모델만 저장할지 여부
                             fp16=True,
                             do_train=True,
                             do_eval=True,
                             # metric_for_best_model
                             # greater_is_better = True,
    )
    trainer = CustomTrainer(model=model,
                            args=args,                                                        # args
                            train_dataset=train_dataset,                                      # 학습데이터
                            eval_dataset=val_dataset,                                         # validation 데이터
                            compute_metrics=compute_metrics,                                  # 모델 평가 방식
                            callbacks=[EarlyStoppingCallback(early_stopping_patience=stop)],) # callback
    trainer.train()
    del model
    del trainer
    gc.collect() # python 자원 관리 
    torch.cuda.empty_cache() # gpu 자원관리   

hidden_layers : 12
now_hidden_layers : 12
Round 0


Some weights of the model checkpoint at snunlp/KR-BERT-char16424 were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be remov

Step,Training Loss,Validation Loss,F1-sum
100,3.991700,2.711256,0.834125
200,3.121100,2.004720,0.907494
300,2.512900,1.640080,0.919167


KeyboardInterrupt: ignored

k-fold 교차 검증에서 성능을 높이기 위해서는 다음과 같은 방법들이 있습니다.

더 많은 데이터 사용: 모델의 성능을 높이는 가장 확실한 방법은 더 많은 데이터를 사용하는 것입니다. 가능하다면 더 많은 데이터를 수집하고, 모델의 성능을 높이기 위해 다양한 데이터 증강 기술을 적용할 수 있습니다.

하이퍼파라미터 튜닝: 하이퍼파라미터를 최적화하여 모델의 성능을 높일 수 있습니다. 이를 위해 grid search, random search, Bayesian optimization 등의 방법을 사용할 수 있습니다.

모델 구조 변경: 모델 구조를 변경하여 성능을 높일 수 있습니다. 예를 들어, 더 깊은 네트워크, 더 넓은 네트워크, 더 복잡한 네트워크 등을 사용할 수 있습니다.

앙상블: 여러 모델의 예측 결과를 결합하여 성능을 높일 수 있습니다. 예를 들어, 다양한 모델 아키텍처를 사용하거나, 다양한 하이퍼파라미터 조합으로 학습한 모델들을 결합할 수 있습니다.

더 나은 평가 지표 사용: 모델의 성능을 높이기 위해 더 나은 평가 지표를 사용할 수 있습니다. 예를 들어, 다양한 클래스에 대한 정확도와 함께 F1-score, AUC 등을 사용할 수 있습니다.

데이터 전처리: 데이터 전처리를 통해 모델의 성능을 높일 수 있습니다. 예를 들어, 데이터 정규화, 특징 추출, 노이즈 제거 등을 수행할 수 있습니다.

모델 가중치 초기화: 모델의 가중치를 초기화하는 방법을 변경하여 모델의 성능을 높일 수 있습니다. 예를 들어, He initialization, Xavier initialization 등을 사용할 수 있습니다.

In [ ]:
def recent_file(path):
    file_name_and_time_lst = []
    # 해당 경로에 있는 파일들의 생성시간을 함께 리스트로 넣어줌. 
    for f_name in os.listdir(f"{path}"):
        written_time = os.path.getctime(f"{path}/{f_name}")
        file_name_and_time_lst.append((f_name, written_time))
    # 생성시간 역순으로 정렬하고, 
    sorted_file_lst = sorted(file_name_and_time_lst, key=lambda x: x[1], reverse=True)
    # 가장 앞에 이는 놈을 넣어준다.
    recent_file = sorted_file_lst[0]
    recent_file_name = recent_file[0]
    return f"{path}/{recent_file_name}"

In [ ]:
gc.collect() # python 자원 관리 
torch.cuda.empty_cache() # gpu 자원관리
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenized = tokenizer(test.문장.tolist(), padding=True, truncation=True, max_length=length, return_tensors="pt")
test_dataset = CustomDataset(tokenized, None)
test_args = TrainingArguments(
    output_dir = './',
    do_train = False,
    do_predict = True,
    per_device_eval_batch_size = 512,   
    dataloader_drop_last = False    
)

tmp = 0
while os.path.isdir(f'fold_{tmp}'):
    tmp += 1

test_results = []
for i in range(tmp):
    print(f'Round {i}')
    # model = AutoModel.from_pretrained(recent_file('custom_model'), config=config)
    model = CustomModel().to(device)
    model.load_state_dict(torch.load(f"{recent_file(f'fold_{i}')}/pytorch_model.bin"))
    trainer = CustomTrainer(
                  model = model, 
                  args = test_args, 
                  compute_metrics = compute_metrics)
    test_results.append(trainer.predict(test_dataset))
    gc.collect() # python 자원 관리 
    torch.cuda.empty_cache() # gpu 자원관리
    del model
    del trainer

Round 0


Some weights of the model checkpoint at snunlp/KR-BERT-char16424 were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
<ipython-input-11-26c33e994149>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTen

In [ ]:
test['유형'] = list(map(lambda x : 유형.inverse_transform([np.argmax(x)]), sum(list(map(lambda x: x.predictions[0], test_results)))/len(test_results)))
test['극성'] = list(map(lambda x : 극성.inverse_transform([np.argmax(x)]), sum(list(map(lambda x: x.predictions[1], test_results)))/len(test_results)))
test['시제'] = list(map(lambda x : 시제.inverse_transform([np.argmax(x)]), sum(list(map(lambda x: x.predictions[2], test_results)))/len(test_results)))
test['확실성'] = list(map(lambda x : 확실성.inverse_transform([np.argmax(x)]), sum(list(map(lambda x: x.predictions[3], test_results)))/len(test_results)))

test['유형'] = list(map(lambda x : x[0], test['유형']))
test['극성'] = list(map(lambda x : x[0], test['극성']))
test['시제'] = list(map(lambda x : x[0], test['시제']))
test['확실성'] = list(map(lambda x : x[0], test['확실성']))

In [ ]:
test['label'] = test['유형'] + '-' + test['극성'] + '-' + test['시제'] + '-' + test['확실성']
test

,ID,문장,유형,극성,시제,확실성,label
0,TEST_0000,장욱진의 가족은 허물 없는 가족애를 처음 공개되는 정약용의 정효자전과 정부인전은 강...,사실형,긍정,현재,확실,사실형-긍정-현재-확실
1,TEST_0001,조지 W 부시 버락 오바마 전 대통령도 전쟁 위험 때문에 버린 카드다,사실형,긍정,현재,확실,사실형-긍정-현재-확실
2,TEST_0002,지난해 1분기 128억원이었던 영업이익이 올해 1분기 505억원으로 급증했다,사실형,긍정,과거,확실,사실형-긍정-과거-확실
3,TEST_0003,수상 작가와 맺으려던 계약서 내용 가운데 일부가 독소 조항으로 해석돼 수정을 요청받...,사실형,긍정,현재,확실,사실형-긍정-현재-확실
4,TEST_0004,결국 최근 KDB산업은행은 대규모 손실 위기에 닥친 에어부산에 140억원 금융지원을...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
...,...,...,...,...,...,...,...
7085,TEST_7085,2020 세계국가편람 모바일 앱은 세계 216개국의 국가개황과 주요 경제지표 사회개...,사실형,긍정,현재,확실,사실형-긍정-현재-확실
7086,TEST_7086,탈세계화 징후들이 반갑지 않은 이유다,추론형,부정,현재,확실,추론형-부정-현재-확실
7087,TEST_7087,틱톡은 6월 인터넷 안전의 달을 맞아 올바른 개인정보 보호 관리 방법 앱 내 유용한...,사실형,긍정,현재,확실,사실형-긍정-현재-확실
7088,TEST_7088,만약 3개월 간 채굴자들의 투표를 거쳐 23 이상의 해시파워가 채굴세 도입에 찬성한...,추론형,긍정,미래,불확실,추론형-긍정-미래-불확실


In [ ]:
sub = pd.read_csv('sample_submission.csv')
sub['label'] = test['label']
tmp = 0
while os.path.exists(f'제출{tmp}.csv'):
    tmp += 1
sub.to_csv(f'제출{tmp}.csv', index=False, mode='w')